In [2]:
import os 
import numpy as np
import re
from dtw import dtw
from tqdm.notebook import tqdm
print("this is test file!")
import seaborn as sns 
from sklearn.cluster import SpectralClustering
import pickle
# np.set_printoptions(threshold=np.inf)
sns.set(color_codes=True)

this is test file!


## 1.根据相似矩阵进行谱聚类的计算 -（需要进行正则化表示）

In [2]:
def normalized_matrix(data):
    data_min = np.min(data.flatten())
    data_max = np.max(data.flatten())
    nor_data = (data_max-data) /(data_max-data_min)
    return nor_data

path = "./processing/DTW_distance_matrix.npy"
DTW_distance = np.load(path, allow_pickle=True)
# print(DTW_similarity)
DTW_similarity = normalized_matrix(DTW_distance)
print(DTW_similarity.shape)
# print(nor_data)


NameError: name 'np' is not defined

### 1.对相似度进行正则化


In [4]:
# 根据邻接矩阵计算谱聚类的结果

save_path = "./processing/DTW_similarity_matrix_clustering.pkl"

cluster_result = {}
rang_clustering = (2, 50)
for i in tqdm(range(rang_clustering[0], rang_clustering[1]+1)):
    n_clusters = i
    seeg_clustering = SpectralClustering(n_clusters=n_clusters, assign_labels="discretize", random_state=0, affinity = "precomputed").fit_predict(DTW_similarity)
#     print(seeg_clustering)
    id_key = "cluster_{}".format(i)
    cluster_result[id_key] = seeg_clustering

with open(save_path, 'wb') as f:
    pickle.dump(cluster_result, f)
    print("Saving result: {}".format(save_path))
    


Saving result: ./processing/DTW_similarity_matrix_clustering.pkl


## 2. 根据谱聚类的结果进行聚类效果的分析

In [3]:
# print(cluster_result['cluster_7'])
cluster_result = np.load("./processing/DTW_similarity_matrix_clustering.pkl", allow_pickle=True)
path_data = "./data/BDP_val_prediction.pkl"
feature_val_prediction = np.load(path_data, allow_pickle=True)
print(feature_val_prediction)
count_ground = [0,0]
count_pre = [0,0]
filter_feature_val_prediction ={}
for id, d in feature_val_prediction.items():
    count_ground[d["ground truth"]] += 1
    count_pre[d["prediction"]] += 1 
    if d["ground truth"] == d["prediction"]:
        filter_feature_val_prediction[id] = d
print("原本的数据分布！")
print("ground_truth:{}  prediction:{}".format(count_ground, count_pre))

count_ground = [0,0]
count_pre = [0,0]
print("预测结果的数据分布！")
for id, d in filter_feature_val_prediction.items():
    count_ground[d["ground truth"]] += 1
    count_pre[d["prediction"]] += 1                    
print("ground_truth:{}  prediction:{}".format(count_ground, count_pre))

{'a5531f6d-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 1}, 'a5c1d8f1-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 0}, 'a5c1d8f4-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 1}, 'a4e5e672-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 0}, 'a59dcf43-fa3a-11e9-a8f2-e0d55e6ff654-2.npy': {'ground truth': 0, 'prediction': 1}, 'b1547040-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'b1546e37-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'b1546f41-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'b12e6658-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 1}, 'b1546f3e-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 0}, 'e076f408-2552-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'prediction': 1}, 'b12e67be-2555-11ea-9699-e0d55e6ff654-0.npy': {'ground truth': 1, 'predicti

In [4]:
from collections import Counter

for id, data in cluster_result.items():
    cluster_data = data
    print(dict(Counter(cluster_data)))
    

{1: 1716, 0: 2508}
{2: 959, 0: 2208, 1: 1057}
{1: 926, 0: 1943, 3: 617, 2: 738}
{2: 760, 3: 1762, 4: 649, 0: 515, 1: 538}
{1: 562, 4: 1482, 0: 899, 3: 438, 5: 705, 2: 138}
{4: 518, 2: 1300, 1: 667, 5: 362, 0: 693, 3: 561, 6: 123}
{5: 356, 1: 1215, 4: 594, 2: 447, 6: 369, 0: 647, 7: 509, 3: 87}
{5: 542, 8: 1092, 3: 538, 0: 652, 6: 328, 4: 365, 1: 453, 7: 108, 2: 146}
{3: 433, 1: 1003, 7: 531, 6: 476, 0: 464, 8: 582, 4: 304, 9: 174, 5: 77, 2: 180}
{6: 519, 1: 958, 10: 468, 3: 226, 7: 427, 0: 583, 4: 229, 9: 224, 8: 297, 2: 41, 5: 252}
{1: 369, 2: 890, 10: 309, 11: 315, 5: 486, 7: 356, 9: 214, 3: 86, 0: 595, 4: 350, 8: 37, 6: 217}
{4: 298, 1: 811, 10: 266, 2: 337, 12: 302, 3: 84, 8: 489, 7: 468, 0: 464, 11: 268, 5: 171, 6: 34, 9: 232}
{6: 333, 3: 775, 8: 235, 12: 358, 1: 376, 0: 474, 4: 457, 9: 61, 2: 158, 11: 298, 10: 262, 5: 230, 7: 26, 13: 181}
{3: 249, 10: 773, 12: 220, 13: 195, 9: 281, 0: 494, 2: 158, 7: 55, 6: 432, 11: 354, 1: 327, 5: 347, 8: 31, 4: 196, 14: 112}
{7: 291, 4: 743, 9:

### 2.1 统计每一个簇中的positive/negative 的占比

In [5]:
# BDP_val_prediction_path = "./data/BDP_val_prediction.pkl"
# BDP_val_prediction = np.load(BDP_val_prediction_path, allow_pickle=True)
# for i in range(5,10):
#     Best_label_cluster = cluster_result["cluster_{}".format(i)]
#     p_n_ratio = {}
#     for id_cluster, fid_list in Best_label_cluster.items():
#         labels_n_p = [0]*2
#         for f_id in fid_list:
#             label = BDP_val_prediction[f_id]['ground truth']
#             labels_n_p[label] += 1
#         p_n_ratio[id_cluster] = labels_n_p
#         
#     for id_cluster, l_n_p in p_n_ratio.items():
#         a = l_n_p[0]/(l_n_p[0]+l_n_p[1])
#         b = l_n_p[1]/(l_n_p[0]+l_n_p[1])
#         print("id_cluster:{} negative:{}/{}, positive:{}/{}".format(id_cluster, l_n_p[0], a, l_n_p[1], b))


    

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [11]:
from sklearn.metrics import davies_bouldin_score
path_data = "./data/feature_true_id_prediction.pkl"
feature_true_id_prediction = np.load(path_data, allow_pickle=True)
feature_true_id_prediction_numpy = []
for id, d in feature_true_id_prediction.items():
    feature_true_id_prediction_numpy.append(d.tolist())
# print(feature_true_id_prediction_numpy)
for n_clusters, d in cluster_result.items(): 
    labels = d
    score = davies_bouldin_score(feature_true_id_prediction_numpy, labels)
    print("{}:{}".format(n_clusters, score))

cluster_2:41.67326190064809
cluster_3:3.388257201238627
cluster_4:2.9092455469403946
cluster_5:2.552813812596579
cluster_6:6.203412186632961
cluster_7:5.7722124243716495
cluster_8:6.197357908341307
cluster_9:3.533183428307999
cluster_10:3.8310209847741263
cluster_11:3.9279787419431695
cluster_12:3.6929278853521743
cluster_13:3.6244128631602024
cluster_14:3.819532011607728
cluster_15:3.8508968307299853
cluster_16:4.007962769805355
cluster_17:3.688042830126387
cluster_18:4.071351404361547
cluster_19:3.7818416227823017
cluster_20:4.271130893748393
cluster_21:3.8383270570986405
cluster_22:6.173000873903714
cluster_23:4.101459739484481
cluster_24:4.575357344612629
cluster_25:4.448047615177031
cluster_26:4.547006193524126
cluster_27:3.99470908985235
cluster_28:4.733995230156857
cluster_29:4.9543928382205085
cluster_30:5.65007336481194
cluster_31:5.552009470872037
cluster_32:4.9708333270882035
cluster_33:4.451443155085523
cluster_34:5.177091761656977
cluster_35:4.780004798675697
cluster_36:5.

### 2.1 选择表现性能最好的簇

In [12]:
# 根据评分选择最好的簇 DBI 得分最低的簇为 cluster=7
cluster_name = "cluster_5"
label_list = cluster_result["cluster_5"]
wave_clusters = {}
wave_clusters_id = {}

# clusters feeature {cluster id :(raw data, raw data id)}
for i ,(id, d)  in enumerate(feature_true_id_prediction.items()):
    label = label_list[i]
    if label not in wave_clusters.keys():
        wave_clusters[label] = []
        wave_clusters_id[label] = []
    wave_clusters[label].append((d, id))
    wave_clusters_id[label].append(id)
# print(wave_clusters)
with open("./processing/wave_clusters_id_feature_data.pkl", 'wb') as f:
    pickle.dump(wave_clusters, f)
    print("Cluster ID-feature data finished!")
with open("./processing/Best_label_cluster.pkl", 'wb') as f:
    pickle.dump(wave_clusters_id, f)
    print("簇滤波id序列构建完成！")

Cluster ID-feature data finished!
簇滤波id序列构建完成！


In [13]:
def similarity_DTW(s1, s2):
    ratio = 50 # 设定的放缩系数，避免数据的相似度过于集中
    euclidean_norm = lambda x, y: np.abs(ratio * (x - y))
    d, cost_matrix, acc_cost_matrix, path = dtw(s1, s2, dist=euclidean_norm)
    score = 1 - np.tanh(d)  # 相似度的评分【0,1】 0： 完全不同， 1： 完全相同
    return score

def get_top_score_series(series_dict, top_n = 5):
    scores = []
    raw_data_series = [x[0] for x in series_dict]
    for s_a, feature_id in tqdm(series_dict):
        score = 0
        for s_b in raw_data_series:
            score += similarity_DTW(s_a, s_b)
        avg_score = score / len(series_dict)
        _ = (feature_id, score)
        scores.append(_)
    result = sorted(scores, key=lambda x:-x[1])
    result = result[:top_n]
    return result

In [14]:
# DBA
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
def sum_of_squares(s,series,cost_mat,delta_mat):
    return sum(map(lambda t:squared_DTW(s,t,cost_mat,delta_mat),series))

def approximate_medoid_index(series,cost_mat,delta_mat):
    if len(series)<=50:
        indices = range(0,len(series))
    else:
        indices = np.random.choice(range(0,len(series)),50,replace=False)

    medoid_ind = -1
    best_ss = 1e20
    for index_candidate in indices:
        candidate = series[index_candidate]
        ss = sum_of_squares(candidate,series,cost_mat,delta_mat)
        if(medoid_ind==-1 or ss<best_ss):
            best_ss = ss
            medoid_ind = index_candidate
    return medoid_ind

def squared_DTW(s,t,cost_mat,delta_mat):
    s_len = len(s)
    t_len = len(t)
    length = len(s)
    fill_delta_mat_dtw(s, t, delta_mat)
    cost_mat[0, 0] = delta_mat[0, 0]
    for i in range(1, s_len):
        cost_mat[i, 0] = cost_mat[i-1, 0]+delta_mat[i, 0]

    for j in range(1, t_len):
        cost_mat[0, j] = cost_mat[0, j-1]+delta_mat[0, j]

    for i in range(1, s_len):
        for j in range(1, t_len):
            diag,left,top =cost_mat[i-1, j-1], cost_mat[i, j-1], cost_mat[i-1, j]
            if(diag <=left):
                if(diag<=top):
                    res = diag
                else:
                    res = top
            else:
                if(left<=top):
                    res = left
                else:
                    res = top
            cost_mat[i, j] = res+delta_mat[i, j]
    return cost_mat[s_len-1,t_len-1]

def fill_delta_mat_dtw(center, s, delta_mat):
    slim = delta_mat[:len(center),:len(s)]
    np.subtract.outer(center, s,out=slim)
    np.square(slim, out=slim)

def DBA_update(center, series, cost_mat, path_mat, delta_mat):
    options_argmin = [(-1, -1), (0, -1), (-1, 0)]
    updated_center = np.zeros(center.shape)
    n_elements = np.array(np.zeros(center.shape), dtype=int)
    center_length = len(center)
    for s in series:
        s_len = len(s)
        fill_delta_mat_dtw(center, s, delta_mat)
        cost_mat[0, 0] = delta_mat[0, 0]
        path_mat[0, 0] = -1

        for i in range(1, center_length):
            cost_mat[i, 0] = cost_mat[i-1, 0]+delta_mat[i, 0]
            path_mat[i, 0] = 2

        for j in range(1, s_len):
            cost_mat[0, j] = cost_mat[0, j-1]+delta_mat[0, j]
            path_mat[0, j] = 1

        for i in range(1, center_length):
            for j in range(1, s_len):
                diag,left,top =cost_mat[i-1, j-1], cost_mat[i, j-1], cost_mat[i-1, j]
                if(diag <=left):
                    if(diag<=top):
                        res = diag
                        path_mat[i,j] = 0
                    else:
                        res = top
                        path_mat[i,j] = 2
                else:
                    if(left<=top):
                        res = left
                        path_mat[i,j] = 1
                    else:
                        res = top
                        path_mat[i,j] = 2

                cost_mat[i, j] = res+delta_mat[i, j]

        i = center_length-1
        j = s_len-1

        while(path_mat[i, j] != -1):
            updated_center[i] += s[j]
            n_elements[i] += 1
            move = options_argmin[path_mat[i, j]]
            i += move[0]
            j += move[1]
        assert(i == 0 and j == 0)
        updated_center[i] += s[j]
        n_elements[i] += 1

    return np.divide(updated_center, n_elements)
def performDBA(series, n_iterations=5):
    n_series = len(series)
    max_length = reduce(max, map(len, series))

    cost_mat = np.zeros((max_length, max_length))
    delta_mat = np.zeros((max_length, max_length))
    path_mat = np.zeros((max_length, max_length), dtype=np.int8)

    medoid_ind = approximate_medoid_index(series,cost_mat,delta_mat)
    center = series[medoid_ind]

    for i in range(0,n_iterations):
        center = DBA_update(center,
                            series, cost_mat, path_mat, delta_mat)

    return center 

In [15]:
# 1.Clustering HeatMap
DBA_id_wave = {}
for cluster_id, data in tqdm(wave_clusters.items()):
    series = []
    for d, id in data:
        series.append(d)
    s_DBA = performDBA(series)
    DBA_id_wave[cluster_id] = s_DBA

with open("./processing/DBA_wave.pkl", 'wb') as f:
    pickle.dump(DBA_id_wave, f)
    print("Saving Success!")







Saving Success!
